In [1]:
import json
import requests

credentials = json.load(open('tmdb_keys.json'))
api_key = credentials['api_key']
api_token = credentials['api_token']

# Piattaforma: [TMDB](https://www.themoviedb.org/)

## Documentazione della Piattaforma

**TMDB** è uno dei più grandi database di Serie TV e Film, costruito e mantenuto dalla propria community. Ogni suo contenuto è pertanto generato dagli utenti, che possono anche recensire, votare e salvare i contenuti stessi.  
 La piattaforma offre un vasto insieme di dati relativi al mondo del cinema e del movie streaming, resi accessibili tramite le proprie API. La documentazione è studiata per supportare sia sviluppatori principianti che esperti, offrendo strumenti e riferimenti utili per un’integrazione efficace.

Le API sono fornite in due versioni, v3 e v4, accessibili a questi link:
* v3: https://developer.themoviedb.org/v3/docs/getting-started
* v4: https://developer.themoviedb.org/v4/docs/getting-started
Rispetto alla versione precedente, la versione 4 si limita a migliorare il meccanismo di Autenticazione degli Utenti. Questa differenza è meglio spiegata nella sezione dedicata.

La versione 3 è ancora tuttavia la versione di riferimento dell'API, motivo per cui in questo report si fa riferimento ad essa, ove non specificato diversamente. 

### Link alla Documentazione

Gli sviluppatori possono iniziare a esplorare le API di TMDB attraverso le seguenti risorse fondamentali:
- **Getting Started**: Il punto di partenza che illustra requisiti, modalità di autenticazione e best practices per l’integrazione. 
    - Link: https://developer.themoviedb.org/docs/getting-started

- **OpenAPI Specification (OAS)**: La specifica completa dell'API secondo lo standard OAS. disponibile per entrambe le versioni dell'API. 
    - Link: https://developer.themoviedb.org/openapi/

- **Daily Exports**: File aggiornati quotidianamente che forniscono un dump di alcuni dati senza richiedere alcuna autenticazione. Sono in particolare resi disponibili gli ID di Film, Serie TV, Persone, Collezioni, TV Networks, Keywords e Case di Produzione.   
    I dati vengono scaricati in formato .gz, possono essere decompressi e letti riga per riga. L'intero file non corrisponde ad un JSON valido, mentre è garantito che ogni riga di esso lo sia.
    - Link: https://developer.themoviedb.org/docs/daily-id-exports
    - Esempio di URL per scaricare gli ID dei film: http://files.tmdb.org/p/exports/movie_ids_03_27_2025.json.gz

- **FAQ** e altri approfondimenti: Sezioni dedicate per chiarire dubbi su limitazioni, utilizzo dei dati e specifiche tecniche, inclusa la documentazione su "Popularity and Trending" per comprendere come vengono calcolati i punteggi e le tendenze dei contenuti.
    - Link: https://developer.themoviedb.org/docs/faq e https://developer.themoviedb.org/docs/popularity-and-trending


TMDB adotta un modello di API RESTful. Le richieste vengono effettuate tramite il protocollo HTTP e le risposte sono solitamente restituite in formato JSON, facilitando così l'integrazione e l'interoperabilità con diverse applicazioni.


### Individuare i Dati Disponibili

Le API di TMDB consentono l’accesso a numerose entità che rappresentano i dati disponibili sulla piattaforma. Questi includono:
- **Film e Serie TV**: Informazioni dettagliate su titoli, generi, date di rilascio e altre caratteristiche.

- **Cast e Crew**: Dati relativi a registi, attori e altri professionisti del settore.

- **Recensioni e Voti**: Feedback degli utenti, punteggi e classifiche.

- **Trending e Popularity**: Endpoints che evidenziano i contenuti più popolari e di tendenza.


#### Profilo Account 

Oltre ai dati inerenti ai contenuti cinematografici, TMDB offre anche endpoint dedicati alla gestione del profilo account. Questi dati, solitamente accessibili nella sezione "Account" dell’API, comprendono:
- Avatar
- Nome e Username
- Descrizione del profilo
- Indicazione sulla visibilità di contenuti per adulti (boolean)
- Liste personalizzate, tra cui:
    - Lista dei Preferiti
    - Watchlist
    - Lista dei Votati
    - Altre liste personalizzate

Questi endpoint permettono agli utenti di gestire le proprie preferenze e di interagire in maniera personalizzata con la piattaforma, facilitando operazioni come il salvataggio di contenuti e la visualizzazione di statistiche personali.

| **Attributo**  | **Tipo**   | **Descrizione**                                                                                                              |
|:---------------|:-----------|:-----------------------------------------------------------------------------------------------------------------------------|
| id             | _integer_  | Identificatore univoco dell'account. Di default 0. utente                                                                    |
| name           | _string_   | Nome impostato dall’utente; può essere visualizzato pubblicamente all’interno della piattaforma. Può essere nullo.           |
| username       | _string_   | Nome univoco scelto dall’utente per l’accesso.                                                                               |
| include_adult  | _boolean_  | Indica se l’account è abilitato a visualizzare contenuti contrassegnati come “per adulti”. Di default è impostato a _true_.  |
| avatar         | _object_   | Oggetto che contiene informazioni sull’avatar dell’utente. Comprende due sottocampi: gravatar e tmdb.                        |
| iso_639_1      | _string_   | Codice della lingua preferita dell'utente.                                                                                   |
| iso_3166_1     | _string_   | Codice del paese dell'utente.                                                                                                |


Di seguito è riportato un esempio:

```python
{
    "avatar": {
        "gravatar": {"hash": "92d7f7061f387cc2da77463666996b6e"},
        "tmdb": {"avatar_path": null},
    },
    "id": 21900582,
    "iso_639_1": "en",
    "iso_3166_1": "IT",
    "name": "",
    "include_adult": true,
    "username": "FilippoCorti",
}
```

#### Relazioni tra Oggetti

Poiché il punto centrale del Portale TMDB consiste nei contenuti generati, le TMDB API non prevedono oggetti che modellino in modo esplicito le relazioni sociali tra gli utenti (come followers, friends, mentions o followings).
Sono invece resi disponibili tutti i dati riguardanti i contenuti (serie tv e film) che gli utenti hanno caricato sulla piattaforma.

Nonostante ciò, anche tramite TMDB è possibile costruire Reti Sociali, che coinvolgono direttamente gli utenti oppure gli attori ed i registi dei contenuti caricati. In seguito indichiamo alcuni spunti per riuscire a costruire Reti basandosi su relazioni implicite, ovvero non espressamente indicate nelle API della Piattaforma ma ottenibili incrociando opportunamente i dati disponibili.

Una forma indiretta di relazione tra utenti può derivare dalla visibilità pubblica di alcune attività: ad esempio, se un utente condivide una lista o una watchlist pubblica, altri utenti possono consultarla e, in qualche caso, interagire con contenuti simili.
Gli utenti registrati possono infatti creare liste personalizzate, aggiungere contenuti alla propria watchlist o segnare film e serie come preferiti. Queste azioni possono riflettere preferenze e interessi personali, ma non generano una vera e propria relazione diretta tra diversi account.   
Similmente, è possibile associare utenti che hanno dato un giudizio simile ad uno stesso film, o che hanno lasciato una recensione sotto la stessa serie tv.

Possiamo trovare relazioni implicite anche tra i contenuti presenti nella piattaforma. Ad esempio, è possibile individuare film che condividono lo stesso cast, in particolare attori o registi, oppure film appartenenti allo stesso genere cinematografico, come azione, commedia o fantascienza. Queste connessioni consentono di analizzare la rete di legami tematici o produttivi all’interno del database.

L’unica forma di relazione esplicita tra account sulla piattaforma TMDB è rappresentata dalle Discussions, in cui gli utenti possono partecipare attivamente rispondendo a thread tematici. Le Discussions, tuttavia, non sono rese accessibili tramite le API ufficiali e, poiché qualsiasi forma di scraping è espressamente vietata dal regolamento della piattaforma, risultano di fatto inaccessibili.


<img src="image-20250404-113849.png" width="" align="" />

#### Contenuti accessibili e meta-dati dei Contenuti
In questa sezione vengono analizzate le principali entità messe a disposizione dalla piattaforma. Le entità considerate includono: film, serie TV, stagioni, episodi, persone, recensioni. Per ogni entità vengono presentati i principali attributi disponibili, il tipo di dato, una breve descrizione compresa di eventuali insiemi di valori di default e range di valori ammessi. In aggiunta a questo viene presentato un esempio esplicativo.

##### 1. Movie


| Attributo               | Tipo              | Descrizione                                                                                                        |
|:------------------------|:------------------|:-------------------------------------------------------------------------------------------------------------------|
| adult                   | boolean           | Indica se il contenuto è destinato a un pubblico adulto                                                            |
| backdrop_path           | string            | Percorso relativo all’immagine di sfondo associata al contenuto                                                    |
| belongs_to_collection   | string            | Nome della collezione (franchise) a cui appartiene il contenuto                                                    |
| budget                  | integer           | Budget stimato per la produzione del film, in dollari americani. Di default 0 se non disponibile.                  |
| genres                  | array of objects  | Elenco dei generi associati al contenuto. Ogni oggetto contiene id (intero) e name (stringa)                       |
| homepage                | string            | URL del sito ufficiale del film                                                                                    |
| id                      | integer           | Identificativo univoco del contenuto all’interno di TMDb. Di default 0 se non disponibile.                         |
| imdb_id                 | string            | Identificativo del contenuto su IMDb, utile per cross-reference                                                    |
| original_language       | string            | Lingua originale del contenuto                                                                                     |
| original_title          | string            | Titolo originale del film (prima della localizzazione).                                                            |
| overview                | string            | Trama o descrizione breve del contenuto                                                                            |
| popularity              | number            | Indicatore di popolarità. Di default 0 se non disponibile.                                                         |
| poster_path             | string            | Percorso relativo all’immagine del poster ufficiale del contenuto                                                  |
| production_companies    | array of objects  | Elenco delle case di produzione coinvolte. Ogni oggetto contiene id, logo_path, name, origin_country.              |
| production_countries    | array of objects  | Elenco dei paesi di produzione. Ogni oggetto contiene iso_3166_1 (codice paese) e name                             |
| release_date            | string            | Data ufficiale di uscita del contenuto.                                                                            |
| revenue                 | integer           | Ricavi ottenuti dal contenuto, in dollari americani. Di default 0 se non disponibile.                              |
| runtime                 | integer           | Durata del contenuto in minuti. Di default 0 se non disponibile.                                                   |
| spoken_languages        | array of objects  | Elenco delle lingue parlate nel contenuto. Ogni oggetto contiene english_name, iso_639_1 e name.                   |
| status                  | string            | Stato di distribuzione del contenuto.                                                                              |
| tagline                 | string            | Frase promozionale o slogan del contenuto.                                                                         |
| title                   | string            | Titolo ufficiale del contenuto.                                                                                    |
| video                   | boolean           | Indica se l’elemento è un video. Di default è impostato a true                                                     |
| vote_average            | number            | Media dei voti assegnati dagli utenti. Si tratta di un valore compreso tra 0 e 10. Di default viene impostato a 0. |
| vote_count              | integer           | Numero totale di voti ricevuti da un film o da una serie TV. Di default viene impostato a 0.                       |

```python
{
  "adult": false,
  "backdrop_path": "/9KbUro9uJHUt5xyorQtjWmGfJDE.jpg",
  "belongs_to_collection": null,
  "budget": 270000000,
  "genres": [
    {
      "id": 10751,
      "name": "Famiglia"
    },
    {
      "id": 14,
      "name": "Fantasy"
    }
  ],
  "homepage": "",
  "id": 447273,
  "imdb_id": "tt6208148",
  "origin_country": [
    "US"
  ],
  "original_language": "en",
  "original_title": "Snow White",
  "overview": "Una rivisitazione musicale live-action del classico film del 1937. La magica avventura musicale torna alla storia senza tempo con gli amati personaggi Mammolo, Dotto, Cucciolo, Brontolo, Gongolo, Pisolo e Eolo.",
  "popularity": 173.1867,
  "poster_path": "/cC0uxvCOCdKggiYZYrf011K1HaE.jpg",
  "production_companies": [
    {
      "id": 2,
      "logo_path": "/wdrCwmRnLFJhEoH8GSfymY85KHT.png",
      "name": "Walt Disney Pictures",
      "origin_country": "US"
    },
    {
      "id": 2527,
      "logo_path": "/osO7TGmlRMistSQ5JZusPhbKUHk.png",
      "name": "Marc Platt Productions",
      "origin_country": "US"
    }
  ],
  "production_countries": [
    {
      "iso_3166_1": "US",
      "name": "United States of America"
    }
  ],
  "release_date": "2025-03-19",
  "revenue": 144119098,
  "runtime": 109,
  "spoken_languages": [
    {
      "english_name": "English",
      "iso_639_1": "en",
      "name": "English"
    },
    {
      "english_name": "Spanish",
      "iso_639_1": "es",
      "name": "Español"
    }
  ],
  "status": "Released",
  "tagline": "",
  "title": "Biancaneve",
  "video": false,
  "vote_average": 4.5,
  "vote_count": 475
}
```

##### 2. TV Series

| Attributo                  | Tipo                  | Descrizione                                                                 |
|----------------------------|-----------------------|-----------------------------------------------------------------------------|
| adult                      | Boolean               | Indica se il contenuto è destinato a un pubblico adulto. Di default è `true`. |
| backdrop_path              | String                | Percorso relativo all’immagine di sfondo della serie.                      |
| created_by                 | Array of Objects      | Creatori della serie. Ogni oggetto contiene: `id` (default 0), `credit_id`, `name`, `gender` (default 0), `profile_path`. |
| episode_run_time           | Array of Integers     | Durata tipica degli episodi in minuti.                                     |
| first_air_date             | String                | Data di prima messa in onda della serie.               |
| genres                     | Array of Objects      | Generi associati alla serie. Ogni oggetto include `id` (default 0) e `name`. |
| homepage                   | String                | URL del sito ufficiale della serie.                                       |
| id                         | Integer               | Identificativo univoco della serie su TMDb. Valore di default: 0.         |
| in_production              | Boolean               | Indica se la serie è ancora in produzione. Di default è `true`.           |
| languages                  | Array of Strings      | Lingue usate nella serie.                              |
| last_air_date              | String                | Data dell'ultima messa in onda della serie.                               |
| last_episode_to_air        | Object                | Informazioni sull’ultimo episodio trasmesso: `id`, `name`, `overview`, `air_date`, `episode_number`, `runtime`, `production_code`, `season_number`, `still_path`, `vote_average`, `vote_count`, `show_id`. |
| next_episode_to_air        | String         | Informazioni sul prossimo episodio, se previsto.                          |
| networks                   | Array of Objects      | Reti televisive che trasmettono la serie. Ogni oggetto ha `id` (default 0), `logo_path`, `name`, `origin_country`. |
| number_of_episodes         | Integer               | Numero totale di episodi. Valore di default: 0.                           |
| name         | String               | Nome della serie in lingua originale.                           |
| number_of_seasons          | Integer               | Numero totale di stagioni. Valore di default: 0.                          |
| origin_country             | Array of Strings      | Paesi di origine della serie.                         |
| original_language          | String                | Lingua originale della serie.                          |
| original_name              | String                | Nome originale della serie.                                               |
| overview                   | String                | Trama o descrizione generale della serie.                                 |
| popularity                 | Number                | Valore indicativo della popolarità della serie. Di default è `0`.        |
| poster_path                | String                | Percorso dell’immagine ufficiale del poster della serie.                 |
| production_companies       | Array of Objects      | Case di produzione. Ogni oggetto ha `id` (default 0), `name`, `logo_path`, `origin_country`. |
| production_countries       | Array of Objects      | Paesi di produzione. Ogni oggetto ha `iso_3166_1` e `name`.              |
| seasons                    | Array of Objects      | Informazioni sulle stagioni. Ogni oggetto ha: `air_date`, `episode_count` (default 0), `id` (default 0), `name`, `overview`, `poster_path`, `season_number` (default 0), `vote_average` (default 0). |
| spoken_languages           | Array of Objects      | Lingue parlate nella serie. Ogni oggetto include `english_name`, `iso_639_1`, `name`. |
| status                     | String                | Stato attuale della serie (es. "Ended").              |
| tagline                    | String                | Frase promozionale associata alla serie.                                  |
| type                       | String                | Tipo di serie (es. "Scripted", "Documentary").                            |
| vote_average               | Number                | Media dei voti assegnati dagli utenti. Si tratta di un valore compreso tra 0 e 10. Di default viene impostato a 0.     |
| vote_count                 | Integer               | Numero totale di voti ricevuti da un film o da una serie TV. Di default viene impostato a 0.                         |


```json
{
  "adult": false,
  "backdrop_path": "/aY7zv2pfk9H0QxaaL3PBjvalbKQ.jpg",
  "created_by": [
    ...,
    {
      "id": 1287248,
      "credit_id": "5dfc96e39824c80013dd32a6",
      "name": "George Kay",
      "original_name": "George Kay",
      "gender": 2,
      "profile_path": null
    },
    ...,
  ],
  "episode_run_time": [],
  "first_air_date": "2021-01-08",
  "genres": [
    {
      "id": 80,
      "name": "Crime"
    },
    {
      "id": 18,
      "name": "Drama"
    },
    {
      "id": 9648,
      "name": "Mystery"
    }
  ],
  "homepage": "https://www.netflix.com/title/80994082",
  "id": 96677,
  "in_production": false,
  "languages": [
    "fr"
  ],
  "last_air_date": "2023-10-05",
  "last_episode_to_air": {
    "id": 4438258,
    "name": "Chapter 7",
    "overview": "A family reunion years in the making comes to pass. With Mariama's help, Assane makes a final play to stop Keller, right his wrongs and go out in style.",
    "vote_average": 7.792,
    "vote_count": 12,
    "air_date": "2023-10-05",
    "episode_number": 7,
    "episode_type": "mid_season",
    "production_code": "",
    "runtime": 52,
    "season_number": 2,
    "show_id": 96677,
    "still_path": "/dd3xE8GzdMNgeFVK3Cavg8ox36r.jpg"
  },
  "name": "Lupin",
  "next_episode_to_air": null,
  "networks": [
    {
      "id": 213,
      "logo_path": "/wwemzKWzjKYJFfCeiB57q3r4Bcm.png",
      "name": "Netflix",
      "origin_country": ""
    }
  ],
  "number_of_episodes": 17,
  "number_of_seasons": 2,
  "origin_country": [
    "FR"
  ],
  "original_language": "fr",
  "original_name": "Lupin",
  "overview": "Inspired by the adventures of Arsène Lupin, gentleman thief Assane Diop sets out to avenge his father for an injustice inflicted by a wealthy family.",
  "popularity": 14.5879,
  "poster_path": "/h6Z2oogE4mJk2uffdtIlLhb0EHx.jpg",
  "production_companies": [
    {
      "id": 9,
      "logo_path": "/nda3dTUYdDrJ6rZqBpYvY865aDv.png",
      "name": "Gaumont",
      "origin_country": "FR"
    }
  ],
  "production_countries": [
    {
      "iso_3166_1": "FR",
      "name": "France"
    }
  ],
  "seasons": [
    {
      "air_date": "2021-01-08",
      "episode_count": 10,
      "id": 157243,
      "name": "Season 1",
      "overview": "",
      "poster_path": "/i4OgrauapMFV21g6hWtMlQs6IpL.jpg",
      "season_number": 1,
      "vote_average": 7.6
    },
    {
      "air_date": "2023-10-05",
      "episode_count": 7,
      "id": 310008,
      "name": "Season 2",
      "overview": "Now in hiding, Assane must learn to live far from his wife and son. With the suffering they endure because of him, Assane can’t stand it any longer and decides to return to Paris to make them a crazy proposal: leave France and start a new life elsewhere. But the ghosts of the past are never far away, and an unexpected return will turn his plans upside down.",
      "poster_path": "/jPe8HRnnqUTQF1MlVEwsqm6TWsw.jpg",
      "season_number": 2,
      "vote_average": 7.5
    }
  ],
  "spoken_languages": [
    {
      "english_name": "French",
      "iso_639_1": "fr",
      "name": "Français"
    }
  ],
  "status": "Canceled",
  "tagline": "",
  "type": "Scripted",
  "vote_average": 7.734,
  "vote_count": 2317
}
```

##### 3. TV Seasons

| Attributo        | Tipo                | Descrizione |
|------------------|---------------------|-------------|
| _id              | String              | Identificatore interno della stagione all'interno del database TMDb. |
| air_date         | String              | Data di prima messa in onda della stagione. |
| episodes         | Array di oggetti    | Lista degli episodi della stagione. Ogni oggetto include: `air_date` (string), `episode_number` (int, default 0), `id` (int, default 0), `name` (string), `overview` (string), `production_code` (string), `runtime` (int, default 0), `season_number` (int, default 0), `show_id` (int, default 0), `still_path` (string), `vote_average` (number, default 0), `vote_count` (int, default 0), `crew` (array), `guest_stars` (array). |
| crew             | Array di oggetti    | Lista dei membri della crew per ogni episodio. Ogni oggetto include: `department` (string), `job` (string), `credit_id` (string), `adult` (boolean, default true), `gender` (int, default 0), `id` (int, default 0), `known_for_department` (string), `name` (string), `original_name` (string), `popularity` (number, default 0), `profile_path` (string). |
| guest_stars      | Array di oggetti    | Guest star presenti negli episodi. Ogni oggetto include: `character` (string), `credit_id` (string), `order` (int, default 0), `adult` (boolean, default true), `gender` (int, default 0), `id` (int, default 0), `known_for_department` (string), `name` (string), `original_name` (string), `popularity` (number, default 0), `profile_path` (string). |
| name             | String              | Nome ufficiale della stagione. |
| overview         | String              | Descrizione generale della stagione. |
| id               | Integer             | Identificativo univoco della stagione all'interno di TMDb. Default: 0. |
| poster_path      | String              | Percorso relativo dell'immagine di copertina della stagione (poster). |
| season_number    | Integer             | Numero progressivo della stagione nella serie. Default: 0. |
| vote_average     | Number              | Valutazione media della stagione da parte degli utenti. Default: 0. |


```json
{
  "_id": "5f1b6fe566a7c30036fc3304",
  "air_date": "2021-01-08",
  "episodes": [
    ...,
    {
      "air_date": "2021-06-11",
      "episode_number": 10,
      "episode_type": "finale",
      "id": 2944656,
      "name": "Capitolo 10",
      "overview": "Con uno stratagemma impareggiabile Assane e Benjamin reclutano e travestono un astuto complice per assicurare Hubert alla giustizia.",
      "production_code": "",
      "runtime": 50,
      "season_number": 1,
      "show_id": 96677,
      "still_path": "/8r8dgHAINYk6SuZvKbjS0df8Rjs.jpg",
      "vote_average": 7.483,
      "vote_count": 29,
      "crew": [
        ...,
        {
          "job": "Story",
          "department": "Writing",
          "credit_id": "60767722d8f44e00417730b5",
          "adult": false,
          "gender": 2,
          "id": 1287248,
          "known_for_department": "Creator",
          "name": "George Kay",
          "original_name": "George Kay",
          "popularity": 0.2469,
          "profile_path": null
        },
        ...
      ],
      "guest_stars": [
        ...,
        {
          "character": "Sofia Belkacem",
          "credit_id": "6005e625e942ee003fe65b7a",
          "order": 559,
          "adult": false,
          "gender": 1,
          "id": 2280476,
          "known_for_department": "Acting",
          "name": "Shirine Boutella",
          "original_name": "Shirine Boutella",
          "popularity": 0.3135,
          "profile_path": "/4qBT877sIjMLycDBmahEpL1nwEk.jpg"
        },
        ...
      ]
    }
  ],
  "name": "Stagione 1",
  "overview": "Ispirandosi alle avventure di Arsenio Lupin, il ladro gentiluomo Assane Diop decide di vendicare il padre per un'ingiustizia subita per colpa di una ricca famiglia.",
  "id": 157243,
  "poster_path": "/pk3ahGmVunewuDfnXxJYr4U5Fck.jpg",
  "season_number": 1,
  "vote_average": 7.6
}
```

##### 4. TV Episodes


| Attributo        | Tipo                | Descrizione |
|------------------|---------------------|-------------|
| air_date         | String              | Data di messa in onda dell’episodio. |
| crew             | Array di oggetti    | Lista dei membri della crew dell’episodio. Ogni oggetto include: `department` (string), `job` (string), `credit_id` (string), `adult` (boolean, default true), `gender` (integer, default 0), `id` (integer, default 0), `known_for_department` (string), `name` (string), `original_name` (string), `popularity` (number, default 0), `profile_path` (string). |
| episode_number   | Integer             | Numero progressivo dell’episodio nella stagione. Default: 0. |
| guest_stars      | Array di oggetti    | Guest star presenti nell’episodio. Ogni oggetto include: `character` (string), `credit_id` (string), `order` (integer, default 0), `adult` (boolean, default true), `gender` (integer, default 0), `id` (integer, default 0), `known_for_department` (string), `name` (string), `original_name` (string), `popularity` (number, default 0), `profile_path` (string). |
| name             | String              | Titolo ufficiale dell’episodio. |
| overview         | String              | Trama o descrizione dell’episodio. |
| id               | Integer             | Identificatore univoco dell’episodio all’interno di TMDb. Default: 0. |
| production_code  | String              | Codice interno di produzione assegnato all’episodio. |
| runtime          | Integer             | Durata in minuti dell’episodio. Default: 0. |
| season_number    | Integer             | Numero della stagione a cui appartiene l’episodio. Default: 0. |
| still_path       | String              | Percorso relativo all’immagine (still) dell’episodio. |
| vote_average     | Number              | Voto medio ricevuto dagli utenti per l’episodio. Default: 0. |
| vote_count       | Integer             | Numero di voti ricevuti per l’episodio. Default: 0. |


```json
{
  "air_date": "2021-01-08",
  "crew": [
    ...,
    {
      "job": "Director",
      "department": "Directing",
      "credit_id": "6005e750cb3084003ce05931",
      "adult": false,
      "gender": 2,
      "id": 18865,
      "known_for_department": "Directing",
      "name": "Louis Leterrier",
      "original_name": "Louis Leterrier",
      "popularity": 0.6132,
      "profile_path": "/bpqqRRyCLeoAup2OAv1Dtm5C8Tn.jpg"
    },
    ...
  ],
  "episode_number": 1,
  "guest_stars": [
    ...,
    {
      "character": "Kevin",
      "credit_id": "6074f4edfb8346006f8a6db2",
      "order": 501,
      "adult": false,
      "gender": 2,
      "id": 2218461,
      "known_for_department": "Acting",
      "name": "Kamel Guenfoud",
      "original_name": "Kamel Guenfoud",
      "popularity": 0.175,
      "profile_path": "/2No4vnSwvQuGaNYoIq3ipJWTh5L.jpg"
    },
    ...
  ],
  "name": "Capitolo 1 - Il collier della regina",
  "overview": "Anni dopo una tragica ingiustizia, Assane vuole regolare i conti e un debito rubando un collier di diamanti... ma il colpo prende una piega inaspettata.",
  "id": 2613188,
  "production_code": "",
  "runtime": 47,
  "season_number": 1,
  "still_path": "/edgLqKlZR1dB73duRSO3fgBqhOP.jpg",
  "vote_average": 8.185,
  "vote_count": 92
}
```

##### 5. Reviews


| Attributo      | Tipo   | Descrizione                         |
|----------------|--------|-------------------------------------|
| author         | string | Nome dell’autore della recensione |
| author_details | object | Oggetto che contiene dettagli aggiuntivi sull’autore tra cui: `name` (nome dell’autore della recensione) `username` (username dell’autore associato al profilo TMDB) `avatar_path` (percorso per l’avatar dell’autore (se disponibile)) `rating` (valutazione numerica assegnata dall’autore (se fornita), con valori compresi tra 0 e 10) |
| content        | string | Contenuto testuale della recensione |
| created_at     | string | Data e ora in cui la recensione è stata creata |
| id             | string | Identificativo univoco della recensione |
| updated_at     | string | Data e ora dell’ultima modifica della recensione |
| url            | string | URL pubblico della recensione sul sito TMDB |



```json
{
    "author": "Peter McGinn",
    "author_details": {
        "name": "Peter McGinn",
        "username": "narrator56",
        "avatar_path": "/tVbrLJzA2RwRlE7dJLJG54UsAkq.jpg",
        "rating": 7
    },
    "content": "Well, with ten other reviews on this site, I am not going to give a blow-by-blow assessment of what I like or don’t like. I am sure other reviews have touched upon hinge I might point out.\r\n\r\nI will only say that I thought it a really good movie: smartly written and nicely acted. I had some trouble following details in places — not because of the complex time structure (though I admit prefer the straight ahead story style of, say, The Godfather) but rather because I couldn’t get volume up loud enough to hear when the characters spoke softly and/or quickly, and without captions I had to wait for the context of conversations to help fill in what I was missing. It was a shame to have such a little thing in the way of a good movie being great.",
    "created_at": "2023-11-14T20:44:12.255Z",
    "id": "6553dc1cd4fe04011b90af5f",
    "updated_at": "2023-11-14T20:44:12.367Z",
    "url": "https://www.themoviedb.org/review/6553dc1cd4fe04011b90af5f"
}
```

##### 6. People

| Attributo            | Tipo              | Descrizione |
|----------------------|-------------------|-------------|
| adult                | Boolean           | Indica se la persona è associata a contenuti per adulti. Valore di default: true. |
| also_known_as        | Array di stringhe | Altri nomi o pseudonimi con cui la persona è conosciuta. |
| biography            | String            | Biografia testuale della persona. |
| birthday             | String            | Data di nascita. |
| deathday             | String            | Data di morte (se presente). |
| gender               | Integer           | Sesso della persona: 0 = non specificato, 1 = femmina, 2 = maschio, 3 = non binario. Default: 0. |
| homepage             | String            | URL del sito web personale, se disponibile. |
| id                   | Integer           | Identificatore univoco della persona in TMDb. Default: 0. |
| imdb_id              | String            | ID della persona su IMDb. |
| known_for_department | String            | Dipartimento per cui la persona è nota. |
| name                 | String            | Nome completo della persona. |
| place_of_birth       | String            | Luogo di nascita. |
| popularity           | Number            | Indice di popolarità calcolato da TMDb. Valore numerico continuo. Default: 0. |
| profile_path         | String            | Percorso relativo all'immagine del profilo. Usato per costruire URL all'immagine. |


```json
{
  "adult": false,
  "also_known_as": [
    "Christopher Edward Nolan",
    "Chris Nolan",
    "Sir Christopher Nolan",
    "Sir Christopher Edward Nolan",
    "کریستوفر نولان"
  ],
  "biography": "Regista, sceneggiatore e produttore cinematografico britannico.\n\nÈ uno dei registi con maggiori incassi nella storia del cinema con oltre 5 miliardi di dollari.",
  "birthday": "1970-07-30",
  "deathday": null,
  "gender": 2,
  "homepage": null,
  "id": 525,
  "imdb_id": "nm0634240",
  "known_for_department": "Directing",
  "name": "Christopher Nolan",
  "place_of_birth": "Westminster, London, England, UK",
  "popularity": 2.7953,
  "profile_path": "/xuAIuYSmsUzKlUMBFGVZaWsY3DZ.jpg"
}
```

## Estrazione dei Dati

In questa sezione vengono esplicitate tutte le procedure per ottenere i dati social prodotti e rilasciati dalla piattaforma.

### Protocollo di Accesso e Autorizzazione all'Accesso ai Dati

La piattaforma mette a disposizione 3 modalità di autenticazione e accesso ai dati:
1. **Application Level Authentication**
2. **User Level Authentication**
3. **Guest Sessions**

#### Application Level Authentication

L'**Autenticazione a Livello di Applicazione** è considerato il meccanismo di default per l'autenticazione. Il procedimento prevede che l'utente registri un'applicazione attraverso un flusso di autenticazione simile a quello di OAuth 2.0 Client Credentials. La registrazione avviene all'URL https://www.themoviedb.org/settings/api/request e richiede che l'utente sia registrato con un account TMDB.

Al termine della registrazione, l'utente riceve:
* Un'**API Key**
* Un **API Read Access Token**

Entrambe queste credenziali possono essere utilizzate per effettuare una richiesta ad un Endpoint:
* L'API Key può essere aggiunta come *Query Parameter* ad una richiesta, sebbene questa modalità di autenticazione è limitata alle chiamate *GET* a partire dalla Versione 4 dell'API.
* L'API Read Access Token è un *Bearer Access Token*, pertanto può essere inserito nell'Header della richiesta per tutte le chiamate. Nonostante il nome, è possibile utilizzare il Read Access Token anche per semplici operazioni di scrittura (rating di un film, salvataggio in una lista, ecc.).
     **Nota:** qualora si utilizzi l'API Read Access Token per operazioni di scrittura, queste verranno eseguite per conto dell'account a cui il token è associato.

La validità di API Key e API Read Access Token può essere verificata con una semplice chiamata all'endpoint https://api.themoviedb.org/3/authentication:


In [9]:
# Validate API Key

response = requests.get(
    url="https://api.themoviedb.org/3/authentication", 
    params={
        'api_key': api_key
    },
    headers={
        "accept": "application/json"
    }
)

response.json()

{'success': True}

In [18]:
# Validate API Read Access Token

response = requests.get(
    url="https://api.themoviedb.org/3/authentication", 
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}"
    }
)

response.json()

{'success': True}

#### User Level Authentication

L'**Autenticazione a Livello Utente** consente di eseguire operazioni di scrittura per conto di altri utenti di TMDB. Questo flusso richiede chiaramente l'autorizzazione di tale utente e funziona in maniera simile all'Authorization Grant Code Flow di OAuth 2.0.

Il flusso è implementato in maniera molto simile nelle versioni 3 e 4 dell'API, tuttavia:
* Nella *Version 3* il flusso termina con l'acquisizione di un **session_id**, da passare come *Query Parameter* nelle richieste di scrittura.
* Nella *Version 4* il flusso termina con l'acquisizione di un **access_token**, che può essere utilizzato esattamente come l'API Read Access Token dell'applicazione.

A scopo dimostrativo, utilizziamo la versione 3 dell'API, in quanto l'attuale versione di default.

Le fasi per l'acquisizione del session_id sono 3:
1. Creazione di un **Request Token**, all'endpoint https://api.themoviedb.org/3/authentication/token/new, avente una validità di 60 minuti dal momento della creazione.
2. Richiesta di **Autorizzazione** all'utente, ridirezionandolo all'URL https://www.themoviedb.org/authenticate/{REQUEST_TOKEN}. Questa richiesta valida il Request Token utilizzato.
3. Creazione del **session_id**, all'endpoint https://api.themoviedb.org/3/authentication/session/new, utilizzando come *Body Parameter* il Request Token precedentemente ottenuto e validato dall'utente.

In seguito è un esempio di implementazione di questo flusso:

In [44]:
# 1. Obtain a Request Token

response = requests.get(
    url="https://api.themoviedb.org/3/authentication/token/new", 
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}"
    }
)

request_token = response.json()['request_token']

print(request_token)
print(f"Redirect to https://www.themoviedb.org/authenticate/{request_token}")

ad96431dfcde3f5fe68bfff206b224c16050970a
Redirect to https://www.themoviedb.org/authenticate/ad96431dfcde3f5fe68bfff206b224c16050970a


2. Redirect the user to https://www.themoviedb.org/authenticate/{request_token} to validate the Request Token. Optionally, you can provide a `redirect_url` just like in OAuth.

| ![](auth_part1.jpg) | ![](auth_part2.jpg) |
|---|---|


In [46]:
# 3. Create the session_id

response = requests.post(
    url="https://api.themoviedb.org/3/authentication/session/new", 
    params={
        'request_token': request_token
    },
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}"
    }
)

if response.json()['success']:
    print(response.json()['session_id'])
    session_id = response.json()['session_id']
else:
    print(response.json()['status_message'])
    session_id = None



662bb2f9b49bf554220ac85c558b8ec945cf8d26


In [48]:
# 4. Use the session_id in a new request

if session_id:
    response = requests.post(
        url="https://api.themoviedb.org/3/account/21900582/favorite",
        params={
            'session_id': session_id
        },
        json={
            "media_type": "movie",
            "media_id": 550,
            "favorite": True
        },
        headers={
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {api_token}"
        }
    )

    print(response.text)

{"success":true,"status_code":12,"status_message":"The item/record was updated successfully."}


#### Guest Sessions

Le **Guest Sessions** sono un meccanismo alternativo di Autenticazione a Livello Utente. 

Consentono la creazione di un **guest_session_id**, senza necessità di autorizzazione da parte di alcun utente, con permessi di scrittura limitati. In particolare, è consentito solamente il rating di film, serie TV ed episodi di serie TV, mentre non possono essere utilizzati guest_session_id per creare liste, aggiungere ai preferiti, ecc..

Il codice per creare un guest_session_id è il seguente:

In [5]:
# Create a Guest Session

response = requests.get(
    url="https://api.themoviedb.org/3/authentication/guest_session/new", 
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}"
    }
)

response.json()

{'success': True,
 'guest_session_id': 'd2268adca9c6ce6eb59f5d6883f74333',
 'expires_at': '2025-03-27 16:25:44 UTC'}

### Endpoint di Accesso ai Dati identificati

In seguito sono specificati alcuni degli Endpoint più significativi, in dettaglio. Il resto degli Endpoint funziona in maniera fondamentalmente equivalente.

#### Movie

##### [GET /3/movie/{movie_id}/account_states](https://developer.themoviedb.org/reference/movie-account-states)

**Parametri**

| Nome                                    | Tipo  | Tipo del parametro | Descrizione                                  |
| --------------------------------------- | ----- | ------------------ | -------------------------------------------- |
| `movie_id` <sup><b>NECESSARIO</b></sup> | path  | _integer_          | Identificativo univoco del contenuto         |
| `session_id`                            | query | _string_           | Identificativo univoco della sessione        |
| `guest_session_id`                      | query | _string_           | Identificativo univoco della sessione ospite |

> NOTA: Se né `session_id` né `guest_session_id` vengono specificati, il risultato restituito sarà relativo all'utente associato al token API utilizzato nella richiesta.

**Risposta**

| Attributo   | Tipo                  | Descrizione                                                                   |
| ----------- | --------------------- | ----------------------------------------------------------------------------- |
| `id`        | _integer_             | Identificativo univoco del contenuto                                          |
| `favorite`  | _boolean_             | Indica se l'oggetto richiesto è tra i preferiti dell'utente                   |
| `rated`     | _boolean_ \| _object_ | Valutazione assegnata dall'utente. `false` se l'utente non ha ancora valutato |
| `watchlist` | _boolean_             | Indica se l'oggetto richiesto è stato aggiunto alla watchlist dell'utente     |

**Oggetto all'interno di `rated`** (se presente)

| Attributo | Tipo     | Descrizione                       |
| --------- | -------- | --------------------------------- |
| `value`   | _number_ | Valutazione assegnata dall’utente |

**Esempi di risposta**

```json
{"id":447273,"favorite":false,"rated":false,"watchlist":false}
{"id":447273,"favorite":true,"rated":false,"watchlist":false}
{"id":447273,"favorite":true,"rated":{"value":7.0},"watchlist":false}
{"id":447273,"favorite":true,"rated":{"value":7.0},"watchlist":true}
```

**Codice**

In [ ]:
movie_id = 447273  # Identificativo univoco del contenuto

response = requests.get(
    f"https://api.themoviedb.org/3/movie/{movie_id}/account_states",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
    params={
        "session_id": session_id,
    },
)

response.json()

##### [GET /3/movie/{movie_id}/alternative_titles](https://developer.themoviedb.org/reference/movie-alternative-titles)

**Parametri**

| Nome                                    | Tipo  | Tipo del parametro | Descrizione                                                           |
| --------------------------------------- | ----- | ------------------ | --------------------------------------------------------------------- |
| `movie_id` <sup><b>NECESSARIO</b></sup> | path  | int                | Identificativo univoco del contenuto                                  |
| `country`                               | query | string             | Codice ISO 3166-1 alpha-2 del paese per filtrare i titoli alternativi |

**Risposta**

| Attributo | Tipo             | Descrizione                                                 |
| --------- | ---------------- | ----------------------------------------------------------- |
| `id`      | integer          | Identificativo univoco del contenuto                        |
| `titles`  | array of objects | Indica se l'oggetto richiesto è tra i preferiti dell'utente |

**Oggetto all'interno di `titles`**

| Attributo    | Tipo   | Descrizione                                     |
| ------------ | ------ | ----------------------------------------------- |
| `iso_3166_1` | string | Codice del paese secondo lo standard ISO 3166-1 |
| `title`      | string | Titolo localizzato del contenuto                |
| `type`       | string | Tipo di titolo (valore definito dall'utente)    |

**Codice**

In [ ]:
movie_id = 447273  # Identificativo univoco del contenuto

response = requests.get(
    f"https://api.themoviedb.org/3/movie/{movie_id}/alternative_titles",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
)

response.json()

##### [GET /3/discover/movie](https://developer.themoviedb.org/reference/discover-movie)

**Parametri**

| Nome             | Tipo   | Tipo del parametro | Descrizione                                                                 |
| ---------------- | ------ | ------------------ | --------------------------------------------------------------------------- |
| `include_adult`  | query  | boolean            | Indica se includere contenuti per adulti (default: `false`)                 |
| `include_video`  | query  | boolean            | Indica se includere video (default: `false`)                                |
| `language`       | query  | string             | Lingua dei risultati (default: `en-US`)                                     |
| `page`           | query  | int                | Numero della pagina dei risultati (default: `1`)                            |
| `sort_by`        | query  | string             | Criterio di ordinamento dei risultati (esempio: `popularity.desc`)          |

(esistono altri parametri di meno rilievo come **Advanced Filtering**)

**Risposta**

| Attributo      | Tipo             | Descrizione                                                                 |
| -------------- | ---------------- | --------------------------------------------------------------------------- |
| `page`         | integer          | Numero della pagina dei risultati                                           |
| `results`      | array of objects | Lista dei film trovati                                                      |
| `adult`        | boolean          | Indica se il film è per adulti                                              |
| `backdrop_path`| string           | Percorso dell'immagine di sfondo                                            |
| `genre_ids`    | array of integers| ID dei generi del film                                                      |
| `id`           | integer          | Identificativo univoco del film                                             |
| `original_language` | string      | Lingua originale del film                                                   |
| `original_title` | string         | Titolo originale del film                                                   |
| `overview`     | string           | Descrizione del film                                                        |
| `popularity`   | float            | Popolarità del film                                                         |
| `poster_path`  | string           | Percorso dell'immagine del poster                                           |
| `release_date` | string           | Data di rilascio del film                                                   |
| `title`        | string           | Titolo del film                                                             |
| `video`        | boolean          | Indica se il film è un video                                                |
| `vote_average` | float            | Voto medio del film                                                         |
| `vote_count`   | integer          | Numero di voti ricevuti dal film                                            |

NB per iterare su più pagine si può seguire la seguente logica:

In [ ]:
base_url = "https://api.themoviedb.org/3/discover/movie"
page = 1
total_pages = 1  # Inizialmente impostato a 1 per entrare nel ciclo

while page <= total_pages:
    
    url = f"{base_url}?include_adult=false&include_video=false&language=en-US&page={page}&sort_by=popularity.desc"
    response = requests.get(url, headers={})
    data = response.json()

    # Stampa i risultati della pagina corrente
    for movie in data['results']:
        print(f"Title: {movie['title']}, Release Date: {movie['release_date']}") # / Una qualsiasi operazione 

    # Aggiorna il numero totale di pagine
    total_pages = data['total_pages']
    page += 1

#### People

##### [GET /3/movie/{movie_id}/credits](https://developer.themoviedb.org/reference/movie-credits)

**Parametri**

| Nome                                    | Tipo  | Tipo del parametro | Descrizione                             |
| --------------------------------------- | ----- | ------------------ | --------------------------------------- |
| `movie_id` <sup><b>NECESSARIO</b></sup> | path  | _integer_          | Identificativo univoco del contenuto    |
| `language`                              | query | _string_           | Lingua dei risultati (default: `en-US`) |

**Risposta**

| Attributo | Tipo               | Descrizione                            |
| --------- | ------------------ | -------------------------------------- |
| `id`      | _integer_          | Identificativo univoco del contenuto   |
| `cast`    | _array of objects_ | Elenco degli attori                    |
| `crew`    | _array of objects_ | Elenco dei membri della troupe tecnica |

**Oggetto all'interno di `cast`**

| Attributo              | Tipo               | Descrizione                                                                |
| ---------------------- | ------------------ | -------------------------------------------------------------------------- |
| `adult`                | _boolean_          | Indica se l'attore è coinvolto in contenuti rivolti a un pubblico adulto   |
| `gender`               | _integer_          | Genere dell’attore (0 = sconosciuto, 1 = donna, 2 = uomo, 3 = non binario) |
| `id`                   | _integer_          | ID univoco della persona                                                   |
| `known_for_department` | _string_           | Reparto per cui è conosciuto                                               |
| `name`                 | _string_           | Nome dell’attore                                                           |
| `original_name`        | _string_           | Nome originale                                                             |
| `popularity`           | _number_           | Indice di popolarità                                                       |
| `profile_path`         | _string_ \| _null_ | Percorso all’immagine del profilo                                          |
| `cast_id`              | _integer_          | ID del cast                                                                |
| `character`            | _string_           | Personaggio interpretato                                                   |
| `credit_id`            | _string_           | ID del credito                                                             |
| `order`                | _integer_          | Ordine di apparizione nel cast                                             |

**Oggetto all'interno di `crew`**

| Attributo              | Tipo               | Descrizione                                                                  |
| ---------------------- | ------------------ | ---------------------------------------------------------------------------- |
| `adult`                | _boolean_          | Indica se la persona è coinvolta in contenuti rivolti a un pubblico adulto   |
| `gender`               | _integer_          | Genere della persona (0 = sconosciuto, 1 = donna, 2 = uomo, 3 = non binario) |
| `id`                   | _integer_          | ID univoco della persona                                                     |
| `known_for_department` | _string_           | Reparto per cui è conosciuto                                                 |
| `name`                 | _string_           | Nome della persona                                                           |
| `original_name`        | _string_           | Nome originale                                                               |
| `popularity`           | _number_           | Indice di popolarità                                                         |
| `profile_path`         | _string_ \| _null_ | Percorso all’immagine del profilo                                            |
| `credit_id`            | _string_           | ID del credito                                                               |
| `department`           | _string_           | Reparto tecnico                                                              |
| `job`                  | _string_           | Ruolo specifico all'interno del reparto                                      |

**Codice**

In [5]:
movie_id = 447273  # Identificativo univoco del contenuto

response = requests.get(
    f"https://api.themoviedb.org/3/movie/{movie_id}/credits",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
)

response.json()

##### [GET /3/tv/{series_id}/season/{season_number}/credits](https://developer.themoviedb.org/reference/tv-season-credits)

| Nome                                         | Tipo  | Tipo del parametro | Descrizione                             |
| -------------------------------------------- | ----- | ------------------ | --------------------------------------- |
| `series_id` <sup><b>NECESSARIO</b></sup>     | path  | _integer_          | Identificativo univoco del contenuto    |
| `season_number` <sup><b>NECESSARIO</b></sup> | path  | _integer_          | Numero della stagione                   |
| `language`                                   | query | _string_           | Lingua dei risultati (default: `en-US`) |

**Risposta**

| Attributo | Tipo               | Descrizione                            |
| --------- | ------------------ | -------------------------------------- |
| `id`      | _integer_          | Identificativo univoco del contenuto   |
| `cast`    | _array of objects_ | Elenco degli attori                    |
| `crew`    | _array of objects_ | Elenco dei membri della troupe tecnica |

**Oggetto all'interno di `cast`**

| Attributo              | Tipo               | Descrizione                                                                |
| ---------------------- | ------------------ | -------------------------------------------------------------------------- |
| `adult`                | _boolean_          | Indica se l'attore è coinvolto in contenuti rivolti a un pubblico adulto   |
| `gender`               | _integer_          | Genere dell’attore (0 = sconosciuto, 1 = donna, 2 = uomo, 3 = non binario) |
| `id`                   | _integer_          | ID univoco della persona                                                   |
| `known_for_department` | _string_           | Reparto per cui è conosciuto                                               |
| `name`                 | _string_           | Nome dell’attore                                                           |
| `original_name`        | _string_           | Nome originale                                                             |
| `popularity`           | _number_           | Indice di popolarità                                                       |
| `profile_path`         | _string_ \| _null_ | Percorso all’immagine del profilo                                          |
| `cast_id`              | _integer_          | ID del cast                                                                |
| `character`            | _string_           | Personaggio interpretato                                                   |
| `credit_id`            | _string_           | ID del credito                                                             |
| `order`                | _integer_          | Ordine di apparizione nel cast                                             |

**Oggetto all'interno di `crew`**

| Attributo              | Tipo               | Descrizione                                                                  |
| ---------------------- | ------------------ | ---------------------------------------------------------------------------- |
| `adult`                | _boolean_          | Indica se la persona è coinvolta in contenuti rivolti a un pubblico adulto   |
| `gender`               | _integer_          | Genere della persona (0 = sconosciuto, 1 = donna, 2 = uomo, 3 = non binario) |
| `id`                   | _integer_          | ID univoco della persona                                                     |
| `known_for_department` | _string_           | Reparto per cui è conosciuto                                                 |
| `name`                 | _string_           | Nome della persona                                                           |
| `original_name`        | _string_           | Nome originale                                                               |
| `popularity`           | _number_           | Indice di popolarità                                                         |
| `profile_path`         | _string_ \| _null_ | Percorso all’immagine del profilo                                            |
| `credit_id`            | _string_           | ID del credito                                                               |
| `department`           | _string_           | Reparto tecnico                                                              |
| `job`                  | _string_           | Ruolo specifico all'interno del reparto                                      |

**Codice**

In [ ]:
series_id = 1396  # Identificativo univoco del contenuto
season_number = 1  # Numero della stagione

response = requests.get(
    f"https://api.themoviedb.org/3/tv/{series_id}/season/{season_number}/credits",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
)

response.json()

##### [GET /3/tv/{series_id}/season/{season_number}/episode/{episode_number}/credits](https://developer.themoviedb.org/reference/tv-episode-credits)

| Nome                                          | Tipo  | Tipo del parametro | Descrizione                             |
| --------------------------------------------- | ----- | ------------------ | --------------------------------------- |
| `series_id` <sup><b>NECESSARIO</b></sup>      | path  | _integer_          | Identificativo univoco del contenuto    |
| `season_number` <sup><b>NECESSARIO</b></sup>  | path  | _integer_          | Numero della stagione                   |
| `episode_number` <sup><b>NECESSARIO</b></sup> | path  | _integer_          | Numero dell'episodio della stagione     |
| `language`                                    | query | _string_           | Lingua dei risultati (default: `en-US`) |

**Risposta**

| Attributo     | Tipo               | Descrizione                            |
| ------------- | ------------------ | -------------------------------------- |
| `id`          | _integer_          | Identificativo univoco del contenuto   |
| `cast`        | _array of objects_ | Elenco degli attori                    |
| `crew`        | _array of objects_ | Elenco dei membri della troupe tecnica |
| `guest_stars` | _array of objects_ | Elenco delle guest star                |

**Oggetto all'interno di `cast` e `guest_stars`**

| Attributo              | Tipo               | Descrizione                                                                |
| ---------------------- | ------------------ | -------------------------------------------------------------------------- |
| `adult`                | _boolean_          | Indica se l'attore è coinvolto in contenuti rivolti a un pubblico adulto   |
| `gender`               | _integer_          | Genere dell’attore (0 = sconosciuto, 1 = donna, 2 = uomo, 3 = non binario) |
| `id`                   | _integer_          | ID univoco della persona                                                   |
| `known_for_department` | _string_           | Reparto per cui è conosciuto                                               |
| `name`                 | _string_           | Nome dell’attore                                                           |
| `original_name`        | _string_           | Nome originale                                                             |
| `popularity`           | _number_           | Indice di popolarità                                                       |
| `profile_path`         | _string_ \| _null_ | Percorso all’immagine del profilo                                          |
| `cast_id`              | _integer_          | ID del cast                                                                |
| `character`            | _string_           | Personaggio interpretato                                                   |
| `credit_id`            | _string_           | ID del credito                                                             |
| `order`                | _integer_          | Ordine di apparizione nel cast                                             |

**Oggetto all'interno di `crew`**

| Attributo              | Tipo               | Descrizione                                                                  |
| ---------------------- | ------------------ | ---------------------------------------------------------------------------- |
| `adult`                | _boolean_          | Indica se la persona è coinvolta in contenuti rivolti a un pubblico adulto   |
| `gender`               | _integer_          | Genere della persona (0 = sconosciuto, 1 = donna, 2 = uomo, 3 = non binario) |
| `id`                   | _integer_          | ID univoco della persona                                                     |
| `known_for_department` | _string_           | Reparto per cui è conosciuto                                                 |
| `name`                 | _string_           | Nome della persona                                                           |
| `original_name`        | _string_           | Nome originale                                                               |
| `popularity`           | _number_           | Indice di popolarità                                                         |
| `profile_path`         | _string_ \| _null_ | Percorso all’immagine del profilo                                            |
| `credit_id`            | _string_           | ID del credito                                                               |
| `department`           | _string_           | Reparto tecnico                                                              |
| `job`                  | _string_           | Ruolo specifico all'interno del reparto                                      |

**Codice**

In [ ]:
series_id = 1396  # Identificativo univoco del contenuto
season_number = 1  # Numero della stagione
episode_number = 1  # Numero dell'episodio

response = requests.get(
    f"https://api.themoviedb.org/3/tv/{series_id}/season/{season_number}/episode/{episode_number}/credits",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
)

response.json()

##### [GET /3/person/{person_id}/movie_credits](https://developer.themoviedb.org/reference/person-movie-credits)

**Parametri**

| Nome                                     | Tipo  | Tipo del parametro | Descrizione                             |
| ---------------------------------------- | ----- | ------------------ | --------------------------------------- |
| `person_id` <sup><b>NECESSARIO</b></sup> | path  | _integer_          | Identificativo univoco della persona    |
| `language`                               | query | _string_           | Lingua dei risultati (default: `en-US`) |

**Risposta**

| Attributo | Tipo               | Descrizione                                                    |
| --------- | ------------------ | -------------------------------------------------------------- |
| `cast`    | _array of objects_ | Elenco dei film in cui la persona ha recitato                  |
| `crew`    | _array of objects_ | Elenco dei film in cui la persona ha lavorato nel team tecnico |

**Oggetto all'interno di `cast`**

| Attributo           | Tipo                | Descrizione                                             |
| ------------------- | ------------------- | ------------------------------------------------------- |
| `adult`             | _boolean_           | Indica se il contenuto è destinato a un pubblico adulto |
| `backdrop_path`     | _string_            | Percorso all’immagine di sfondo del film                |
| `genre_ids`         | _array of integers_ | Elenco degli ID dei generi associati al film            |
| `id`                | _integer_           | Identificativo univoco del film                         |
| `original_language` | _string_            | Lingua originale del contenuto                          |
| `original_title`    | _string_            | Titolo originale del film                               |
| `overview`          | _string_            | Descrizione del contenuto                               |
| `popularity`        | _number_            | Indice di popolarità                                    |
| `poster_path`       | _string_            | Percorso all’immagine del poster                        |
| `release_date`      | _string_            | Data di uscita                                          |
| `title`             | _string_            | Titolo del film                                         |
| `video`             | _boolean_           | Indica se è un video                                    |
| `vote_average`      | _number_            | Media dei voti ricevuti                                 |
| `vote_count`        | _integer_           | Numero totale di voti                                   |
| `character`         | _string_            | Personaggio interpretato                                |
| `credit_id`         | _string_            | ID del credito                                          |
| `order`             | _integer_           | Ordine di apparizione nel cast                          |

**Oggetto all'interno di `crew`**

| Attributo           | Tipo                | Descrizione                                             |
| ------------------- | ------------------- | ------------------------------------------------------- |
| `adult`             | _boolean_           | Indica se il contenuto è destinato a un pubblico adulto |
| `backdrop_path`     | _string_            | Percorso all’immagine di sfondo del film                |
| `genre_ids`         | _array of integers_ | Elenco degli ID dei generi associati al film            |
| `id`                | _integer_           | Identificativo univoco del film                         |
| `original_language` | _string_            | Lingua originale del contenuto                          |
| `original_title`    | _string_            | Titolo originale del film                               |
| `overview`          | _string_            | Descrizione del contenuto                               |
| `popularity`        | _number_            | Indice di popolarità                                    |
| `poster_path`       | _string_            | Percorso all’immagine del poster                        |
| `release_date`      | _string_            | Data di uscita                                          |
| `title`             | _string_            | Titolo del film                                         |
| `video`             | _boolean_           | Indica se è un video                                    |
| `vote_average`      | _number_            | Media dei voti ricevuti                                 |
| `vote_count`        | _integer_           | Numero totale di voti                                   |
| `credit_id`         | _string_            | ID del credito                                          |
| `department`        | _string_            | Reparto tecnico di appartenenza                         |
| `job`               | _string_            | Ruolo specifico svolto all'interno del film             |

**Codice**

In [ ]:
person_id = 17419  # Identificativo univoco della persona

response = requests.get(
    f"https://api.themoviedb.org/3/person/{person_id}/movie_credits",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
)

response.json()

#### Reviews

##### [GET /3/movie/{movie_id}/reviews](https://developer.themoviedb.org/reference/movie-reviews)

**Parametri**

| Nome                                    | Tipo  | Tipo del parametro | Descrizione                                   |
| --------------------------------------- | ----- | ------------------ | --------------------------------------------- |
| `movie_id` <sup><b>NECESSARIO</b></sup> | path  | _integer_          | Identificativo univoco del contenuto          |
| `language`                              | query | _string_           | Lingua dei risultati (default: `en-US`)       |
| `page`                                  | query | _integer_          | Numero di pagina dei risultati (default: `1`) |

**Risposta**

> NOTA: il limite di risultati per pagina è 20.

| Attributo       | Tipo               | Descrizione                          |
| --------------- | ------------------ | ------------------------------------ |
| `id`            | _integer_          | Identificativo univoco del contenuto |
| `page`          | _integer_          | Numero della pagina restituita       |
| `results`       | _array of objects_ | Elenco delle recensioni dell'utente  |
| `total_pages`   | _integer_          | Numero totale di pagine              |
| `total_results` | _integer_          | Numero totale di recensioni          |

**Oggetto all'interno di `results`**

| Attributo        | Tipo     | Descrizione                       |
| ---------------- | -------- | --------------------------------- |
| `author`         | _string_ | Nome dell’autore della recensione |
| `author_details` | _object_ | Dettagli dell'autore              |
| `content`        | _string_ | Testo della recensione            |
| `created_at`     | _string_ | Data di creazione                 |
| `id`             | _string_ | Identificativo della recensione   |
| `updated_at`     | _string_ | Data di ultima modifica           |
| `url`            | _string_ | URL della recensione su TMDB      |

**Oggetto all'interno di `author_details`**

| Attributo     | Tipo               | Descrizione                      |
| ------------- | ------------------ | -------------------------------- |
| `name`        | _string_           | Nome completo dell'autore        |
| `username`    | _string_           | Nome utente dell'autore          |
| `avatar_path` | _string_ \| _null_ | Percorso dell’avatar dell’autore |
| `rating`      | _string_ \| _null_ | Valutazione assegnata            |

**Codice**

In [5]:
movie_id = 447273  # Identificativo univoco del contenuto

response = requests.get(
    f"https://api.themoviedb.org/3/movie/{movie_id}/reviews",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
)

response.json()

##### [GET /3/tv/{series_id}/reviews](https://developer.themoviedb.org/reference/tv-series-reviews)

**Parametri**

| Nome                                     | Tipo  | Tipo del parametro | Descrizione                                   |
| ---------------------------------------- | ----- | ------------------ | --------------------------------------------- |
| `series_id` <sup><b>NECESSARIO</b></sup> | path  | _integer_          | Identificativo univoco della serie            |
| `language`                               | query | _string_           | Lingua dei risultati (default: `en-US`)       |
| `page`                                   | query | _integer_          | Numero di pagina dei risultati (default: `1`) |

**Risposta**

> NOTA: il limite di risultati per pagina è 20.

| Attributo       | Tipo               | Descrizione                        |
| --------------- | ------------------ | ---------------------------------- |
| `id`            | _integer_          | Identificativo univoco della serie |
| `page`          | _integer_          | Numero della pagina restituita     |
| `results`       | _array of objects_ | Elenco delle recensioni            |
| `total_pages`   | _integer_          | Numero totale di pagine            |
| `total_results` | _integer_          | Numero totale di recensioni        |

**Oggetto all'interno di `results`**

| Attributo        | Tipo     | Descrizione                       |
| ---------------- | -------- | --------------------------------- |
| `author`         | _string_ | Nome dell’autore della recensione |
| `author_details` | _object_ | Dettagli dell'autore              |
| `content`        | _string_ | Testo della recensione            |
| `created_at`     | _string_ | Data di creazione                 |
| `id`             | _string_ | Identificativo della recensione   |
| `updated_at`     | _string_ | Data di ultima modifica           |
| `url`            | _string_ | URL della recensione su TMDB      |

**Oggetto all'interno di `author_details`**

| Attributo     | Tipo               | Descrizione                      |
| ------------- | ------------------ | -------------------------------- |
| `name`        | _string_           | Nome completo dell'autore        |
| `username`    | _string_           | Nome utente dell'autore          |
| `avatar_path` | _string_ \| _null_ | Percorso dell’avatar dell’autore |
| `rating`      | _string_ \| _null_ | Valutazione assegnata            |

**Codice**

In [1]:
series_id = 1396  # Identificativo univoco del contenuto

response = requests.get(
    f"https://api.themoviedb.org/3/tv/{series_id}/reviews",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {api_token}",
    },
)

response.json()

## Software

### Librerie per la Piattaforma
La documentazione fornisce indicazioni su strumenti e librerie, sia ufficiali che di terze parti, compatibili con i principali linguaggi di programmazione. Troviamo l'elenco completo di queste librerie e wrapper, divisi appunto per linguaggi di programmazione, a questo link: https://developer.themoviedb.org/docs/wrappers-and-libraries

Per Python ad esempio, sono disponibili queste librerie/wrapper di terze parti :
- **tmdbsimple** by celiao
- **tmdb3** by Toilal
- **pytmdb3** by wagner
- **themoviedb** by doganaydin


Esempio di ricerca usando la funzione Search() resa disponibile nel wrapper **tmdbsimple**


Per questo progetto non abbiamo usato nessuna libreria ma direttamente le API fornite dalla piattaforma.

In [8]:
import tmdbsimple as tmdb
tmdb.API_KEY = api_key
search = tmdb.Search()
response = search.movie(query='Inception')

for movie in search.results[:3]:
    print(f"Titolo: {movie['title']}")
    print(f"Data di uscita: {movie['release_date']}")
    print(f"Voto medio: {movie['vote_average']}")
   
    print("-----------------------")

Titolo: Inception
Data di uscita: 2010-07-15
Voto medio: 8.4
-----------------------
Titolo: The Crack: Inception
Data di uscita: 2019-10-04
Voto medio: 6.7
-----------------------
Titolo: Inception: The Cobol Job
Data di uscita: 2010-12-07
Voto medio: 7.3
-----------------------
